# **1. Importaciones**

In [43]:
import pandas as pd

In [44]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# **2. Extracción del Archivo Tratado**

In [45]:
df = pd.read_csv('/content/drive/MyDrive/ONE/HACKATHON/DATASET/DATASET_EDA/dataset_limpio_final.csv')

In [46]:
df.head()

,antiguedad,plan,metodo_pago,facturas_impagas,frecuencia_uso,tickets_soporte,tipo_contrato,cambios_plan,canal_adquisicion,churn
0,16,estandar,transferencia_bancaria,2,15,0,mensual,3,call_center,0
1,120,estandar,efectivo,3,2,10,mensual,2,referido,1
2,29,premium,tarjeta_credito,2,28,1,anual,3,web,0
3,48,estandar,tarjeta_debito,0,9,2,mensual,1,referido,0
4,13,estandar,tarjeta_credito,1,19,5,mensual,0,web,0


# **3. Feature Enginnering**

## "Fricción del servicio" = cuántos tickets abre el cliente POR CADA DÍA que usa el servicio

In [47]:

# NUEVAS FEATURES

# 0. Feature Fricción del servicio
df['friccion_del_servicio'] = df['tickets_soporte'] / (df['frecuencia_uso'] + 1)



# 1. Engagement Score (combina uso + antigüedad + estabilidad)
df['engagement_score'] = (
    (df['frecuencia_uso'] / 30) * 0.4 +        # % de uso mensual
    (df['antiguedad'] / 120) * 0.3 +           # % de antigüedad máxima
    (1 - df['cambios_plan'] / 5) * 0.3         # Estabilidad (menos cambios = más estable)
)

# 2. Valor del plan (necesario para las siguientes features)
def insertar_valor_plan(x):
    if x == 'basico':
        return 4.99
    elif x == 'estandar':
        return 9.99
    else:
        return 14.99

df['valor_plan_num'] = df['plan'].apply(insertar_valor_plan)

# 3. Ratio Valor/Uso (cuánto paga por día que usa)
df['ratio_valor_uso'] = df['valor_plan_num'] / (df['frecuencia_uso'] + 1)

# 4. Riesgo Financiero (deuda acumulada)
df['riesgo_financiero'] = df['facturas_impagas'] * df['valor_plan_num']

# 5. Cliente Problemático (muchos tickets + poco uso)
df['cliente_problematico'] = ((df['tickets_soporte'] > 5) & (df['frecuencia_uso'] < 10)).astype(int)

# 6. Early Churn Risk (nuevo + ya tiene problemas)
df['early_churn_risk'] = ((df['antiguedad'] < 6) & (df['facturas_impagas'] > 0)).astype(int)

# 7. Plan Premium + Contrato Mensual (combinación de riesgo)
df['premium_mensual'] = ((df['plan'] == 'premium') & (df['tipo_contrato'] == 'mensual')).astype(int)


# ORDEN FINAL DE COLUMNAS (ACTUALIZADO)

nuevo_orden = [
    # Contractuales originales (9)
    'antiguedad',
    'plan',
    'metodo_pago',
    'tipo_contrato',
    'frecuencia_uso',
    'cambios_plan',
    'facturas_impagas',
    'tickets_soporte',
    'canal_adquisicion',
    # Features Engineerizadas (8)
    'friccion_del_servicio',
    'engagement_score',
    'valor_plan_num',
    'ratio_valor_uso',
    'riesgo_financiero',
    'cliente_problematico',
    'early_churn_risk',
    'premium_mensual',
    # Target
    'churn'
]

df = df[nuevo_orden]


# VERIFICACIÓN

print(f"Shape final: {df.shape}")  # Debe ser (4242, 18)
print(f"\nColumnas finales ({len(df.columns)}):")
print(df.columns.tolist())

# Ver primeras filas
print("\nPrimeras filas del dataset actualizado:")
print(df.head())

# Estadísticas de las nuevas features
print("\nEstadísticas de las nuevas features:")
print(df[['friccion_del_servicio', 'engagement_score', 'ratio_valor_uso',
          'riesgo_financiero', 'cliente_problematico', 'early_churn_risk',
          'premium_mensual']].describe())


Shape final: (4251, 18)

Columnas finales (18):
['antiguedad', 'plan', 'metodo_pago', 'tipo_contrato', 'frecuencia_uso', 'cambios_plan', 'facturas_impagas', 'tickets_soporte', 'canal_adquisicion', 'friccion_del_servicio', 'engagement_score', 'valor_plan_num', 'ratio_valor_uso', 'riesgo_financiero', 'cliente_problematico', 'early_churn_risk', 'premium_mensual', 'churn']

Primeras filas del dataset actualizado:
   antiguedad      plan             metodo_pago tipo_contrato  frecuencia_uso  \
0          16  estandar  transferencia_bancaria       mensual              15   
1         120  estandar                efectivo       mensual               2   
2          29   premium         tarjeta_credito         anual              28   
3          48  estandar          tarjeta_debito       mensual               9   
4          13  estandar         tarjeta_credito       mensual              19   

   cambios_plan  facturas_impagas  tickets_soporte canal_adquisicion  \
0             3             

In [48]:
df.head()

,antiguedad,plan,metodo_pago,tipo_contrato,frecuencia_uso,cambios_plan,facturas_impagas,tickets_soporte,canal_adquisicion,friccion_del_servicio,engagement_score,valor_plan_num,ratio_valor_uso,riesgo_financiero,cliente_problematico,early_churn_risk,premium_mensual,churn
0,16,estandar,transferencia_bancaria,mensual,15,3,2,0,call_center,0.000000,0.360000,9.99,0.624375,19.98,0,0,0,0
1,120,estandar,efectivo,mensual,2,2,3,10,referido,3.333333,0.506667,9.99,3.330000,29.97,1,0,0,1
2,29,premium,tarjeta_credito,anual,28,3,2,1,web,0.034483,0.565833,14.99,0.516897,29.98,0,0,0,0
3,48,estandar,tarjeta_debito,mensual,9,1,0,2,referido,0.200000,0.480000,9.99,0.999000,0.00,0,0,0,0
4,13,estandar,tarjeta_credito,mensual,19,0,1,5,web,0.250000,0.585833,9.99,0.499500,9.99,0,0,0,0


In [49]:
df.shape

(4251, 18)

# **6. Guardando Dataset_con nueva columna creada**

In [50]:
df.to_csv('dataset_feature_enginnering_final.csv', index=False)